In [3]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, find_state_extract, directory_scratch, directory_project


In [4]:
plate_number = 721
plate = get_postion_number(plate_number)
# plate = 29
print(plate)
# directory = directory_scratch
directory = directory_project
listdir = os.listdir(directory)
list_dir_interest = [name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
dates_datetime = get_dates_datetime(directory,plate)
len(list_dir_interest)

38


271

In [3]:
plate_number = get_plate_number(plate,dates_datetime[0])
print(0,plate_number)
for i in range(len(list_dir_interest)):
    new_plate_number = get_plate_number(plate,dates_datetime[i])
    if plate_number!=new_plate_number:
        plate_number=new_plate_number
        print(i,plate_number)


0 59
61 137
106 76
131 721


In [5]:
begin = 131
end = begin + 60
print('begin =', dates_datetime[begin], '\n  end =', dates_datetime[end])

begin = 2021-08-18 13:24:00 
  end = 2021-08-23 15:24:00


In [66]:
find_state(plate, begin, end, directory,True)

'skeletonization is complete'

In [9]:
find_state_extract(plate, begin, end, directory)

('/Analysis/nx_graph_pruned_labeled.p',
 [(datetime.datetime(2021, 8, 18, 13, 24), 131),
  (datetime.datetime(2021, 8, 18, 15, 25), 132),
  (datetime.datetime(2021, 8, 18, 17, 21), 133),
  (datetime.datetime(2021, 8, 18, 19, 22), 134),
  (datetime.datetime(2021, 8, 18, 21, 22), 135),
  (datetime.datetime(2021, 8, 18, 23, 22), 136),
  (datetime.datetime(2021, 8, 19, 1, 22), 137),
  (datetime.datetime(2021, 8, 19, 3, 22), 138),
  (datetime.datetime(2021, 8, 19, 7, 22), 139),
  (datetime.datetime(2021, 8, 19, 9, 22), 140),
  (datetime.datetime(2021, 8, 19, 11, 23), 141),
  (datetime.datetime(2021, 8, 19, 13, 23), 142),
  (datetime.datetime(2021, 8, 19, 15, 23), 143),
  (datetime.datetime(2021, 8, 19, 17, 23), 144),
  (datetime.datetime(2021, 8, 19, 19, 23), 145),
  (datetime.datetime(2021, 8, 19, 21, 23), 146),
  (datetime.datetime(2021, 8, 19, 23, 23), 147),
  (datetime.datetime(2021, 8, 20, 1, 23), 148),
  (datetime.datetime(2021, 8, 20, 3, 23), 149),
  (datetime.datetime(2021, 8, 20, 5

In [4]:
start = begin + 60
finish = start +2
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[start:finish+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_pruned_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

In [5]:
plt.close('all')
start = 0
finish = start +2
for i in range(start,finish):
    plot_t_tp1([],[],None,None,skels[i],ims[i])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
start = begin + 40
finish = start +2
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[start:finish+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5, 5), np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap = directory + directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_realigned_compressed.mat')
    skel = skel_info['skeleton']
#     skels.append(skel)
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)
plt.close('all')
start = 0
finish = start + 1
for i in range(start, finish):
    plot_t_tp1([],[],None,None,skels[i],ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Create graphs***

In [5]:
num_parallel = 20
time = '30:00'
args=[plate, directory]
run_parallel('extract_nx_graph.py',args,begin,end,num_parallel,time,'extract_nx')

***Extract Width***

In [16]:
num_parallel = 16
time = '5:00:00'
skip = True
args = [plate,directory,skip]
run_parallel('extract_width.py',args,begin,end,num_parallel,time,'extract_width')

***Identify Nodes***

In [4]:
num_parallel = 1
time = '12:00:00'
args=[plate,begin,end, directory]
run_parallel('extract_nodes.py',args,0,0,num_parallel,time,'node_id')

***Hyphae extraction***

In [13]:
num_parallel = 1
time = '5:00:00'
args = [plate, begin, end, directory]
run_parallel('hyphae_extraction.py',args,pd.DataFrame(),num_parallel,time,'hyphae')

In [16]:
 0.2 * 210 * 10**14 *30 *1.8

2.268e+17